In [76]:
## Import Libraries
import pandas as pd
import numpy as np
import calendar
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from adjustText import adjust_text

In [77]:
## Load cleaned data
df = pd.read_csv('Permits_by_ELEM_Boundary_Cleaned.csv')

# Sort df by school in NAME column
df = df.sort_values(by='NAME')

# Remove decimal places from DU and convert to integer
df['DU'] = df['DU'].fillna(0).astype(int)

df

,NAME,B_NAME,CATEGORY,ADDRESS_1,YEAR,MONTH,DU,PLAN,BLG_TYPE,CONTRACTOR,DESCRIPTION
0,Assumption Catholic Elementary School,Assumption School,Elementary,80 BARRETTE ST,2016.0,April,1,101,Single,OTTAWA GENERAL CONTRACTORS,Interior alterations to the kitchen and exteri...
101,Assumption Catholic Elementary School,Assumption School,Elementary,280 LEVIS AVE,2021.0,May,3,238,Apartment,CONTRACTOR UNKNOWN,Construct a 3 storey triplex
102,Assumption Catholic Elementary School,Assumption School,Elementary,10 MCARTHUR AVE,2021.0,May,10,239,Apartment,CONTRACTOR UNKNOWN,Construct a 3 storey apartment building (10 Un...
103,Assumption Catholic Elementary School,Assumption School,Elementary,263 GREENSWAY AVE,2021.0,June,74,NaN,Apartment,CONTRACTOR UNKNOWN,Construct a 6 storey apartment building with u...
104,Assumption Catholic Elementary School,Assumption School,Elementary,163 LONGPRE ST,2021.0,July,2,471,Semi - Detached,CONTRACTOR UNKNOWN,Construct a 3 storey semi-detached dwelling wi...
...,...,...,...,...,...,...,...,...,...,...,...
19939,Thomas D'Arcy McGee Catholic Elementary School,Thomas d'Arcy McGee School,Elementary,567 TAKAMOSE PRIV,2020.0,September,20,4M1581,Stacked Rowhouse,MATTAMY DEVELOPMENT CORPORATION,Construct a 3 storey block of 20 stacked dwell...
19938,Thomas D'Arcy McGee Catholic Elementary School,Thomas d'Arcy McGee School,Elementary,545 TAKAMOSE PRIV,2020.0,September,20,4M-1581,Stacked Rowhouse,MATTAMY HOMES LIMITED,Construct a 3 storey 20 unit stacked dwelling
19937,Thomas D'Arcy McGee Catholic Elementary School,Thomas d'Arcy McGee School,Elementary,523 TAKAMOSE PRIV,2020.0,September,20,4M-1581,Stacked Rowhouse,MATTAMY (ROCKCLIFFE) INC.,Construct a 3 storey 20 unit stacked dwelling
19951,Thomas D'Arcy McGee Catholic Elementary School,Thomas d'Arcy McGee School,Elementary,507 PIMIWIDON ST,2021.0,January,4,4M-1651,Rowhouse,UNIFORM URBAN DEVELOPMENTS LTD.,Construct a 2 storey block of 4 rowhouse dwell...


In [78]:
# Pivot the data to have years as columns and schools as rows
du_pivot = df.pivot_table(values='DU', index='NAME', columns='YEAR', aggfunc='sum', fill_value=0)

# Display the pivoted table
du_pivot

YEAR,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0,2024.0
NAME,,,,,,,,,
Assumption Catholic Elementary School,17,200,105,255,43,189,549,80,20
Chapel Hill Catholic Elementary School,3,0,2,5,0,34,3,0,0
Convent Glen Catholic Elementary School,2,8,7,10,4,0,20,3,0
Corpus Christi Catholic Elementary School,730,221,606,676,1298,679,1036,66,6
Divine Infant Catholic Elementary School,95,218,86,275,84,282,433,29,1
...,...,...,...,...,...,...,...,...,...
St. Rose of Lima Catholic Elementary School,0,0,1,0,1,2,0,1,0
St. Stephen Catholic Elementary School,154,302,330,114,7,310,266,86,2
St. Theresa Catholic Elementary School,24,17,36,31,27,57,278,45,1


In [79]:
def analyze_permit_data(df):
    """
    Analyzes permit data by grouping it by name, year, and month, and calculating the sum of DU.
    Then, it creates a pivot table with the sum of DU values for each name and year, grouped by month.
    The pivot table is then reindexed to have the columns in the desired order.
    
    Parameters:
    - df (DataFrame): The input DataFrame containing the permit data.
    
    Returns:
    - du_by_month_pivot (DataFrame): The pivot table with the corrected order of columns.
    """
    
    # Group the data by name, year, and month and calculate the sum of DU
    du_by_month = df.groupby(['NAME', 'YEAR', 'MONTH'])['DU'].sum().reset_index()
    
    # Create a pivot table with the sum of DU values for each name and year, grouped by month
    du_by_month_pivot = du_by_month.pivot_table(values='DU', index=['NAME', 'YEAR'], columns='MONTH', aggfunc='sum', fill_value=0)
    
    # Calculate the total sum of DU for each name and year
    du_by_month_pivot['Total'] = du_by_month_pivot.sum(axis=1)
    
    # Define the desired order of the columns
    desired_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'Total']
    
    # Reindex the columns of the du_by_month_pivot DataFrame
    du_by_month_pivot = du_by_month_pivot.reindex(columns=desired_order)
    
    return du_by_month_pivot

# Call the analyze_permit_data function
du_by_month_pivot = analyze_permit_data(df)

# Round the values in the pivot table to 0 decimal places
du_by_month_pivot = du_by_month_pivot.round(0)

# Display the pivot table with the corrected order of columns
du_by_month_pivot.head(50)

MONTH                                                 January  February  \
NAME                                          YEAR                        
Assumption Catholic Elementary School         2016.0        0         0   
                                              2017.0        0         0   
                                              2018.0        1         0   
                                              2019.0        0         0   
                                              2020.0        3         2   
                                              2021.0        0         5   
                                              2022.0        0         4   
                                              2023.0        0        13   
                                              2024.0       20         0   
Chapel Hill Catholic Elementary School        2016.0        0         0   
                                              2018.0        0         0   
                                              2019.0        0         0   
                                              2021.0        0         0   
                                              2022.0        0         0   
Convent Glen Catholic Elementary School       2016.0        0         0   
                                              2017.0        0         1   
                                              2018.0        0         0   
                                              2019.0        0         1   
                                              2020.0        0         0   
                                              2022.0        1         0   
                                              2023.0        0         0   
Corpus Christi Catholic Elementary School     2016.0        8        14   
                                              2017.0       15        10   
                                              2018.0      255       124   
                                              2019.0        3         5   
                                              2020.0      177         2   
                                              2021.0        0         0   
                                              2022.0      197         0   
                                              2023.0        1         0   
                                              2024.0        6         0   
Divine Infant Catholic Elementary School      2016.0        9         8   
                                              2017.0        6         9   
                                              2018.0        0         1   
                                              2019.0        0         2   
                                              2020.0        2         1   
                                              2021.0       10       176   
                                              2022.0        2         7   
                                              2023.0        1         0   
                                              2024.0        1         0   
Dr. F. J. McDonald Catholic Elementary School 2016.0        0         0   
                                              2017.0        1         0   
                                              2018.0        0         1   
                                              2019.0        0         0   
                                              2020.0        0         0   
                                              2021.0        2         0   
                                              2022.0        1         1   
                                              2023.0        0         1   
Georges Vanier Catholic Elementary School     2016.0       12         7   
                                              2018.0        0         0   
                                              2019.0        0        78   

MONTH                                                 March  April  May  June  \
NAME              

In [80]:
# Set the style and fonts
plt.style.use('seaborn-whitegrid')
sns.set_context('talk')
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial']
plt.rcParams['text.color'] = '#333333'

# Custom color for the plot
economist_blue = '#4671A2'

# Define the complete range of years
years = np.arange(2016, 2025)  # 2025 is not included, so this is 2016-2024

# Create a DataFrame to hold the full range of years, ensuring each school has entries for all years
full_years_df = pd.DataFrame(years, columns=['YEAR'])

# Create a PDF file to save the figures
with PdfPages('school_du_reports.pdf') as pdf:
    # Iterate over each school in the DataFrame
    for school in df['NAME'].unique():
        # Filter the DataFrame for the current school
        school_df = df[df['NAME'] == school]

        # Aggregate DU values by YEAR to ensure unique YEAR entries
        agg_school_df = school_df.groupby('YEAR')['DU'].sum().reset_index()

        # Merge aggregated data with the full years DataFrame to ensure all years are represented
        merged_df = pd.merge(full_years_df, agg_school_df, on='YEAR', how='left')
        merged_df.fillna(0, inplace=True)  # Fill missing DU values with 0

        # Plot the data as a bar chart
        plt.figure(figsize=(10, 6))
        plt.bar(merged_df['YEAR'], merged_df['DU'], color=economist_blue)

        # Add data labels
        for x, y in zip(merged_df['YEAR'], merged_df['DU']):
            plt.text(x, y, f'{int(y)}', ha='center', va='bottom')

        # Set title and labels
        plt.title(f'Sum of DU by YEAR - {school}', fontsize=20, color='#333333')
        plt.xlabel('YEAR', fontsize=16, color='#333333')
        plt.ylabel('Sum of DU', fontsize=16, color='#333333')
        
        # Customize ticks
        plt.xticks(years, rotation=45, fontsize=14, color='#333333')
        plt.yticks(fontsize=14, color='#333333')
        
        # Customize gridlines
        plt.grid(True, which='major', linestyle='--', linewidth=0.5, color='grey', axis='y')
        
        # Instead of showing the plot, save it to the PDF
        pdf.savefig()  # saves the current figure into the pdf
        plt.close()  # close the figure to free memory

C:\Users\e38282\AppData\Local\Temp\ipykernel_15324\1937739335.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [ ]:
# Save the dataframe to an Excel workbook
#with pd.ExcelWriter('Permits by Elementary Boundary.xlsx') as writer:
#    du_by_month_pivot.to_excel(writer, sheet_name='Summary')
